In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.datasets import imdb

In [3]:
# 1. Tải dữ liệu IMDB
max_words = 10000  # Số lượng từ tối đa được sử dụng
top_words = 5000   # Chỉ giữ lại 5000 từ phổ biến nhất
max_length = 500   # Độ dài tối đa của mỗi đoạn văn bản

In [4]:
print("\nTải dữ liệu IMDB...")
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=top_words)
print(f"Số lượng mẫu huấn luyện: {len(x_train)}, Số lượng mẫu kiểm tra: {len(x_test)}")


Tải dữ liệu IMDB...
17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Số lượng mẫu huấn luyện: 25000, Số lượng mẫu kiểm tra: 25000


In [5]:
# 2. Tiền xử lý dữ liệu (padding để đảm bảo độ dài cố định)
x_train = pad_sequences(x_train, maxlen=max_length)
x_test = pad_sequences(x_test, maxlen=max_length)
print("Dữ liệu đã được padding.")

Dữ liệu đã được padding.


In [6]:
# 3. Xây dựng mô hình LSTM
model = keras.Sequential([
    keras.layers.Embedding(top_words, 128, input_length=max_length),  # Lớp nhúng từ (Embedding)
    keras.layers.LSTM(100, dropout=0.2, recurrent_dropout=0.2),  # Lớp LSTM chính
    keras.layers.Dense(1, activation='sigmoid')  # Lớp đầu ra nhị phân
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [7]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [8]:
# 4. Huấn luyện mô hình
epochs = 3
batch_size = 64
print("\nBắt đầu huấn luyện mô hình...")
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=epochs, batch_size=batch_size, verbose=1)


Bắt đầu huấn luyện mô hình...
Epoch 1/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 632s 2s/step - accuracy: 0.6925 - loss: 0.5607 - val_accuracy: 0.8379 - val_loss: 0.3888
Epoch 2/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 683s 2s/step - accuracy: 0.8263 - loss: 0.4045 - val_accuracy: 0.8338 - val_loss: 0.3872
Epoch 3/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 680s 2s/step - accuracy: 0.8602 - loss: 0.3292 - val_accuracy: 0.8295 - val_loss: 0.3911


In [9]:
# 5. Đánh giá mô hình
print("\nĐánh giá mô hình trên tập kiểm tra...")
loss, accuracy = model.evaluate(x_test, y_test, verbose=1)
print(f"Độ chính xác trên tập kiểm tra: {accuracy * 100:.2f}%")


Đánh giá mô hình trên tập kiểm tra...
782/782 ━━━━━━━━━━━━━━━━━━━━ 137s 175ms/step - accuracy: 0.8261 - loss: 0.3952
Độ chính xác trên tập kiểm tra: 82.95%


In [10]:
# 6. Kiểm tra mô hình với một số đánh giá tùy chỉnh
def predict_review(review, model, word_index):
    reverse_word_index = {v: k for k, v in word_index.items()}  # Tạo lại từ điển từ chỉ mục
    encoded_review = [word_index.get(word, 2) for word in review.split()]  # Chuyển đổi từ sang chỉ mục
    padded_review = pad_sequences([encoded_review], maxlen=max_length)  # Padding
    prediction = model.predict(padded_review)[0][0]  # Dự đoán
    return "Tích cực" if prediction > 0.5 else "Tiêu cực"

In [11]:
# Lấy từ điển từ IMDB
word_index = imdb.get_word_index()

1641221/1641221 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [12]:
# Kiểm tra với một số câu đánh giá
test_reviews = [
    "this movie was fantastic i really loved it",  # Tích cực
    "the film was horrible and boring",  # Tiêu cực
    "it was an excellent movie with great acting",  # Tích cực
    "the plot was dull and the characters were uninteresting"  # Tiêu cực
]

In [13]:
for review in test_reviews:
    sentiment = predict_review(review, model, word_index)
    print(f"Đánh giá: '{review}' -> {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 878ms/step
Đánh giá: 'this movie was fantastic i really loved it' -> Tích cực
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 193ms/step
Đánh giá: 'the film was horrible and boring' -> Tích cực
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 186ms/step
Đánh giá: 'it was an excellent movie with great acting' -> Tích cực
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
Đánh giá: 'the plot was dull and the characters were uninteresting' -> Tích cực
